# Teste brreg API
- <https://data.brreg.no/enhetsregisteret/oppslag/enheter?kommunenummer=1103,1108&fraAntallAnsatte=10&tilAntallAnsatte=50000&sort=navn.norwegian,asc>

In [1]:
import requests
import pandas as pd
from brreg.enhetsregisteret import Client



In [58]:
df = pd.read_json('enheter_sandnes_stavanger.json')
df['naeringskode_1'] = df['naeringskode1'].apply(lambda x: x['kode'])
df['naeringsbeskrivelse_1'] = df['naeringskode1'].apply(lambda x: x['beskrivelse'])
df['organisasjonsform'] = df['organisasjonsform'].apply(lambda x: x['beskrivelse'])
#need to handle nan value for  naeringskode2 and 3
# Function to extract 'kode' from dictionary or return None for NaN
def extract_kode(x):
    if isinstance(x, dict):
        return x['kode']
    else:
        return None

# Function to extract 'beskrivelse' from dictionary or return None for NaN
def extract_beskrivelse(x):
    if isinstance(x, dict):
        return x['beskrivelse']
    else:
        return None
df['naeringskode_2'] = df['naeringskode2'].apply(extract_kode)
df['naeringsbeskrivelse_2'] = df['naeringskode2'].apply(extract_beskrivelse)
df['naeringskode_3'] = df['naeringskode3'].apply(extract_kode)
df['naeringsbeskrivelse_3'] = df['naeringskode3'].apply(extract_beskrivelse)
df['kommune'] = df['forretningsadresse'].apply(lambda x: x['kommune'])
df['forretningsadresse'] = df['forretningsadresse'].apply(lambda x: f"{', '.join(x['adresse'])}, {x['postnummer']} {x['poststed']}")
#change format of overordnet_enhet to text and remove decimal from float representation
df['overordnetEnhet'] = df['overordnetEnhet'].astype(str).apply(lambda x: x.rstrip('.0'))
df['organisasjonsnummer'] = df['organisasjonsnummer'].astype(str)

## Select columns for companies to store in csv

## Create data for company codes and prepare for relationships

In [74]:
business_1 = df[['naeringskode_1', 'naeringsbeskrivelse_1']].drop_duplicates()
business_2 = df[['naeringskode_2', 'naeringsbeskrivelse_2']].drop_duplicates()
business_3 = df[['naeringskode_3', 'naeringsbeskrivelse_3']].drop_duplicates()
business_2.columns = business_1.columns
business_3.columns = business_1.columns
business = pd.concat([business_1, business_2, business_3])

# Check for and remove duplicates
business = business.drop_duplicates()
# Remove _1 from column names   
business.columns = business.columns.str.replace('_1', '')
business.to_csv('csv_files/business_codes.csv', index=False)

## Company relationships

In [100]:
df[['organisasjonsnummer','naeringskode_1']].to_csv('csv_files/rel_business_1.csv', index=False)
df[df['naeringskode_2'].notna()][['organisasjonsnummer','naeringskode_2']].to_csv('csv_files/rel_business_2.csv', index=False)
df[df['naeringskode_3'].notna()][['organisasjonsnummer','naeringskode_3']].to_csv('csv_files/rel_business_3.csv', index=False)

In [106]:
df[-['naeringskode1']]

TypeError: bad operand type for unary -: 'list'

# Look at overordnet enhet

In [ ]:
df[df['overordnetEnhet'].notna()]['overordnetEnhet']

In [12]:
# /enhetsregisteret/api/enheter/{orgnr}
# Define the API endpoint URL
url = "https://data.brreg.no/enhetsregisteret/oppslag/enheter/974760673/roller"

# Define query parameters
#params = {"orgnr": 974760673}

# Make an HTTP GET request to the API
response = requests.get(url, params=params)

In [22]:
# Define the API endpoint URL
orgnr = '915501680'
url = f'https://data.brreg.no/enhetsregisteret/api/enheter/{orgnr}/roller'

# Define headers for the request
headers = {
    'Accept': 'application/json'
}

# Make an HTTP GET request to the API with the specified headers
response = requests.get(url, headers=headers)
data = response.json()

In [40]:
# Initialize empty lists to store data
names = []
roles = []
dates_of_birth = []

# Loop through the JSON response data and extract the desired information
for role_group in data['rollegrupper']:
    role_group_description = role_group['type']['beskrivelse']
    for role in role_group['roller']:
        person = role.get('person', {})  # Check if 'person' key exists, and if not, use an empty dictionary
        name = role['enhet']['navn'][0]
        # Check if "date_of_birth" is available (for a person)
        if 'person' in data:
            date_of_birth = data['person'].get('fodselsdato', '')
        else:
            date_of_birth = ''
        # Append data to lists
        names.append(name)
        roles.append(role_group_description)
        dates_of_birth.append(date_of_birth)

# Create a Pandas DataFrame
df = pd.DataFrame({
    'name': names,
    'role': roles,
    'date_of_birth': dates_of_birth
})

# Display the DataFrame
print(df)

KeyError: 'enhet'

In [41]:
person

{'fodselsdato': '1980-02-16',
 'navn': {'fornavn': 'Andreas',
  'mellomnavn': 'Egge',
  'etternavn': 'Thorsheim'},
 'erDoed': False}

In [28]:
for role_group in data['rollegrupper']:
    role_group_description = role_group['type']['beskrivelse']
    for role in role_group['roller']:
        person = role.get('person', {}) 

In [38]:
role['enhet']['navn'][0]

'AIDER ACCOUNTING AS'

In [10]:

client = Client()
enhet = client.get_enhet('915501680')

In [11]:
enhet

Enhet(organisasjonsnummer='915501680', navn='OTOVO ASA', organisasjonsform=Organisasjonsform(kode='ASA', beskrivelse='Allmennaksjeselskap', utgaatt=None), hjemmeside='www.otovo.no', postadresse=None, registreringsdato_enhetsregisteret=datetime.date(2015, 7, 10), registrert_i_mvaregisteret=True, frivillig_mva_registrert_beskrivelser=[], naeringskode1=Naeringskode(kode='46.180', beskrivelse='Agenturhandel med spesialisert vareutvalg ellers'), naeringskode2=None, naeringskode3=None, hjelpeenhetskode=None, antall_ansatte=91, har_registrert_antall_ansatte=True, overordnet_enhet=None, forretningsadresse=Adresse(adresse=['Torggata 7'], postnummer='0181', poststed='OSLO', kommunenummer='0301', kommune='OSLO', landkode='NO', land='Norge'), stiftelsesdato=datetime.date(2015, 6, 1), institusjonell_sektorkode=InstitusjonellSektorkode(kode='2100', beskrivelse='Private aksjeselskaper mv.'), registrert_i_foretaksregisteret=True, registrert_i_stiftelsesregisteret=False, registrert_i_frivillighetsregis